In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

/Users/wiradharmakencanaputra/miniconda3/envs/jcopml/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


# Import Data

In [2]:
df = pd.read_csv("data/train.csv", index_col="ID")
df.head()

GRE  TOEFL  University_Rating  SOP  LOR  CGPA  Research  Chance_of_Admit
ID                                                                           
250  321    111                  3  3.5  4.0  8.83         1             0.77
434  316    111                  4  4.0  5.0  8.54         0             0.71
20   303    102                  3  3.5  3.0  8.50         0             0.62
323  314    107                  2  2.5  4.0  8.27         0             0.72
333  308    106                  3  3.5  2.5  8.21         1             0.75

# Dataset Splitting

In [3]:
X = df.drop(columns="Chance_of_Admit")
y = df.Chance_of_Admit

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((320, 7), (80, 7), (320,), (80,))

# Training

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from jcopml.tuning import random_search_params as rsp

In [6]:
X_train.head()

GRE  TOEFL  University_Rating  SOP  LOR  CGPA  Research
ID                                                          
323  314    107                  2  2.5  4.0  8.27         0
118  290    104                  4  2.0  2.5  7.46         0
312  328    108                  4  4.5  4.0  9.18         1
255  321    114                  4  4.0  5.0  9.12         0
289  314    104                  4  5.0  5.0  9.02         0

In [9]:
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(), ["GRE", "TOEFL", "SOP", "LOR", "CGPA"]),
    ('categoric', cat_pipe(encoder='onehot'), ["Research", "University_Rating"]),
])

pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', RandomForestRegressor(n_jobs=-1, random_state=42))
])

model = RandomizedSearchCV(pipeline, rsp.rf_params, cv=3, n_iter=50, n_jobs=-1, verbose=1, random_state=42)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   50.6s finished


{'algo__max_depth': 46, 'algo__max_features': 0.4756699028339012, 'algo__min_samples_leaf': 5, 'algo__n_estimators': 196}
0.8741603724498904 0.8024911709698469 0.8128959211565071


# Submission

In [11]:
def submit(model, filename="wira__v1.csv"):
    df_submit = pd.read_csv("data/test.csv", index_col="ID")
    df_submit['Chance_of_Admit'] = model.predict(df_submit)
    df_submit[['Chance_of_Admit']].to_csv(filename, index_label='ID')

In [12]:
submit(model, filename="wira__v1.csv")